<p style="font-family: Arial; font-size:3.75em;color:purple; font-style:bold"><br>
LIGHT TABLE - GENERATION</p><br>

This notebook generate Light Tables based on nexus full-simulations of light, generating a DataFrame that should be used by DetSim.

TAREAS POR IMPLEMENTAR:

* Descripcion de lo que hace cada una de las funciones y sus parametros
* Leer el setup de un fichero de configuracion
* Anyadir tests
* Posibilidad de muestrear los volumenes / areas con un patron asimetrico (mas denso a radios altos, para reflejar mejor los cambios que se producen cerca de los bordes).
* Hacer el setting de IC y de NEXUS desde las funciones, no desde el terminal
* Anyadir tests de que las simulaciones han corrido bien en las colas.

In [1]:
from IPython.core.display import HTML
css = open('css/style-table.css').read() + open('css/style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [92]:
# General importings
import os
import pandas as pd

from math   import ceil

# Specific IC stuff
import invisible_cities.core.system_of_units  as units

# Light Table stuff
from sim_functions       import make_init_file
from sim_functions       import make_config_file
from sim_functions       import run_sims
from sim_functions       import get_num_photons

from table_functions     import get_table_positions
from table_functions     import get_working_paths
from table_functions     import build_table
from table_functions     import get_fnames
from table_functions     import get_table_fname
from table_functions     import get_detector_dimensions

In [3]:
%load_ext autoreload
%autoreload 2

# GENERAL DATA

In [4]:
VERBOSITY = True

In [5]:
MAX_PHOTONS_PER_EVT = 1000000

In [6]:
# Valid options
VALID_DETECTORS    = ["NEXT_NEW", "NEXT100", "NEXT_FLEX"]
VALID_TABLE_TYPES  = ["energy", "tracking"]
VALID_SIGNAL_TYPES = ["S1", "S2"]

# SETUP DATA

In [76]:
RUN_SIMULATIONS = False
GENERATE_TABLE  = False

In [77]:
det_name = "NEXT_NEW"
assert det_name in VALID_DETECTORS, "Wrong Geometry"

In [78]:
table_type = "energy"
assert table_type in VALID_TABLE_TYPES, "Wrong Table Type"

In [79]:
signal_type = "S2"
assert signal_type in VALID_SIGNAL_TYPES, "Wrong Signal Type"

In [80]:
# Typically PmtR11410 for energy tables and SiPM for tracking ones
sensor_name = "PmtR11410"
#sensor_name = "SiPM"
#sensor_name = "TP_SiPM"

In [81]:
pitch = (100.0 * units.mm, 100.0 * units.mm, 200.0 * units.mm)
#pitch = (20.0 * units.mm, 20.0 * units.mm, 20.0 * units.mm)
#pitch = (1.0 * units.mm, 1.0 * units.mm, 1.0 * units.mm)

if((table_type == "tracking") and (pitch[2] > 2.0 * units.mm)):
    print("\n#### WARNING #### - Pitch Z unusually big.")

In [82]:
photons_per_point = 100000

In [83]:
points_per_job = 1

#### Preliminary job

In [84]:
### Getting (photons / point) & (events / point) & (photons / event)
events_per_point  = 1
photons_per_event = photons_per_point

if photons_per_event > MAX_PHOTONS_PER_EVT:
    events_per_point  = ceil(photons_per_point / MAX_PHOTONS_PER_EVT)
    photons_per_event = MAX_PHOTONS_PER_EVT
    photons_per_point = events_per_point * photons_per_event

In [85]:
### Getting Table positions
table_positions = get_table_positions(det_name, table_type, signal_type, pitch)
num_points = len(table_positions)

In [86]:
### Getting Num of jobs
num_jobs = ceil(num_points / points_per_job)
photons_per_job = points_per_job * photons_per_point

In [87]:
### Getting PATHS
config_path, log_path, dst_path, table_path = get_working_paths(det_name)

In [88]:
### Verbosity
if VERBOSITY:
    print(f"\n***** Generating {det_name} Light Table  *****\n")
    print(f"*** Type: {table_type}  -  Signal: {signal_type}  -  Sensor: {sensor_name}")
    print(f"*** Pitch: {pitch} mm")
    print(f"*** Photons/Point = {photons_per_point:.1e} splitted into ...")
    print(f"***    {events_per_point} Events/Point x {photons_per_event:.1e} Photons/Event")
    print(f"*** Total number of points: {num_points:6}")
    #print(table_positions)
    print(f"*** Total number of jobs:   {num_jobs:6}")
    print(f"*** Photons/Job: {photons_per_job}  ->  {photons_per_job/60.e4:.3} minutes/job (@ Harvard)")
    print(f"*** Config PATH: {config_path}")
    print(f"*** Log    PATH: {log_path}")
    print(f"*** Dst    PATH: {dst_path}")
    print(f"*** Table  PATH: {table_path}")


***** Generating NEXT_NEW Light Table  *****

*** Type: energy  -  Signal: S2  -  Sensor: PmtR11410
*** Pitch: (100.0, 100.0, 200.0) mm
*** Photons/Point = 1.0e+05 splitted into ...
***    1 Events/Point x 1.0e+05 Photons/Event
*** Total number of points:     11
*** Total number of jobs:        3
*** Photons/Job: 500000  ->  0.833 minutes/job (@ Harvard)
*** Config PATH: /Users/Javi/Development/NextLightTable/data/NEXT_NEW/config/
*** Log    PATH: /Users/Javi/Development/NextLightTable/data/NEXT_NEW/log/
*** Dst    PATH: /Users/Javi/Development/NextLightTable/data/NEXT_NEW/dst/
*** Table  PATH: /Users/Javi/Development/NextLightTable/data/NEXT_NEW/table/


In [89]:
### Checks
assert num_jobs < 10000, "Number of jobs too high"
assert (photons_per_job/1.e4) < 12*60*60, "Jobs larger than half day"

# LIGHT SIMULATIONS

In [91]:
if RUN_SIMULATIONS:

    print(f"\n*** Running {num_points} Light Simulations ...\n")

    for job_id in range(num_jobs):
        
        # fnames to be run in current job
        init_fnames = []
        log_fnames  = []
        
        # Identifing current job positions
        ini_pos_id = job_id * points_per_job
        for pos in table_positions[ini_pos_id : ini_pos_id + points_per_job]:
            print(f"* Position {pos} - {photons_per_point} photons")
            
            # file names
            init_fname, config_fname, log_fname, dst_fname = get_fnames(det_name ,pos)

            # Check if the sim is already run with the correct num_photons
            if os.path.isfile(dst_fname + '.h5'):
                if (get_num_photons(dst_fname + '.h5') >= photons_per_point):
                    print("  Simulation run previously, so skipping ...")
                    continue
                else:
                    print("  Simulation run previously with less events, so re-running ...")

            # Preparing this position to be simulated
            make_init_file(det_name, init_fname, config_fname)
            
            make_config_file(det_name, config_fname, dst_fname,
                             pos[0], pos[1], pos[2],
                             photons_per_event)
            
            # Adding file names to be run
            init_fnames += [init_fname]
            log_fnames  += [log_fname]
            
        # Launching simulation job
        if(len(init_fnames)):
            print(f"* Submitting job {job_id} with {len(init_fnames)} points\n")
            run_sims(init_fnames, log_fnames, events_per_point)
        else:
            print(f"* Job {job_id} with no points, so skipped\n")


*** Running 11 Light Simulations ...

* Position (-108, -108, 0) - 100000 photons
* Position (-108, -8, 0) - 100000 photons
  Simulation run previously with less events, so re-running ...
* Position (-108, 92, 0) - 100000 photons
  Simulation run previously with less events, so re-running ...
* Position (-8, -108, 0) - 100000 photons
  Simulation run previously with less events, so re-running ...
* Position (-8, -8, 0) - 100000 photons
  Simulation run previously, so skipping ...
* Submitting job 0 with 4 points



AssertionError: When running locally, points_per_job MUST BE 1

# TABLE GENERATION

In [ ]:
if GENERATE_TABLE:

    dimensions = get_detector_dimensions(det_name)
    light_table_fname = table_path + get_table_fname(det_name, table_type,
                                                     signal_type, sensor_name)
    
    # Light Table
    light_table = build_table(det_name, table_type, signal_type, sensor_name, pitch)
    
    light_table.to_hdf(light_table_fname, '/LightTable', mode   = 'w',
                       format = 'table', data_columns = True)

    # Config Table
    config_columns =  ['parameter', 'value']

    config_data    = [['detector' ,            det_name],
                      ['ACTIVE_rad',           str(dimensions['ACTIVE_radius'])],
                      ['ACTIVE_length',        str(dimensions['ACTIVE_length'])],
                      ['EL_GAP',               str(dimensions['EL_gap'])],
                      ['reference_sensor_id',  str(dimensions['ref_sensor'][0])],
                      ['table_type',           table_type],
                      ['signal_type',          signal_type],
                      ['sensor',               sensor_name],
                      ['pitch_x',              str(pitch[0])],
                      ['pitch_y',              str(pitch[1])],
                      ['pitch_z',              str(pitch[2])],
                      ['photons_per_point',    str(photons_per_point)],
                      ['photons_per_event',    str(photons_per_event)],
                      ['events_per_point',     str(events_per_point)],
                      ['total_points',         str(num_points)],
                      ['table_path',           table_path],
                      ['dst_path',             dst_path],
                      ['config_path',          config_path],
                      ['log_path',             log_path]]

    config_table = pd.DataFrame(config_data, columns = config_columns)
    config_table.set_index("parameter", inplace = True)
    
    config_table.to_hdf(light_table_fname, '/Config', mode   = 'a',
                        format = 'table', data_columns = True)

    # Verbosing
    print(f"\n*** Storing Light Table in {light_table_fname} ...\n")